In [1]:
"""
Purpose: To download a platinum neuron mesh


"""

'\nPurpose: To download a platinum neuron mesh\n\n\n'

In [2]:
import datajoint as dj
import numpy as np

In [3]:
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [4]:
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')
schema = dj.schema("microns_minnie65_01")

In [5]:
m65.ProofreadLabel()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,decimation_ratio percentage that the faces are decimated,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
96631955273149705,0,1.00,computer_Auto,spapadop,2019-11-07 13:39:22,=BLOB=,=BLOB=,=BLOB=,partial
107816118160698192,0,1.00,computer_Auto,spapadop,2019-11-07 13:40:50,=BLOB=,=BLOB=,=BLOB=,partial


In [6]:
#combine all the meshes into one mesh
def add_mesh_piece(main_mesh_vertices,main_mesh_faces,sub_mesh_vertices,sub_mesh_faces):
    """
    Purpose: Takes in a large mesh piece and an array of other meshes and 
    returns a large mesh with all meshes appended
    
    Parameters:
    main_mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates
    main_mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices
    sub_mesh_vertices(list of np.arrays) : list of np arrays with the vertices arrays for all subsegments to be added
    sub_mesh_faces(list of np.arrays) : list of np arrays with the faces arrays for all subsegments to be added
    
    Returns:
    mesh_vertices (np.array) : np array store the vertices as rows and the elements as the coordinates for NEW CONCATENATED MESH
    mesh_faces (np.array) : np array store the faces as rows and the elements as the referenced vertices for NEW CONCATENATED MESH
    
    
    Pseudocode: 
    - Checks: 
    a. Make sure there sub_mesh arrays are greater than 0 and of the same length

    1) Count the number of vertices and faces in the main mesh
    2) Iterate through the submesh vertices and faces. In loop:
    a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
    b. Add the vertices_count and add that to every number in the faces array
    c. Concatenate the submesh faces onto the larger mesh face
    d. Save this new vertices and faces as the main_mesh verts and faces
    e. Print out how many new vertices and faces added
    3) Print out number of segments added, total faces/vertices for new mesh
    4) Return the main mesh vertices and faces
    
    """
    #a. Make sure there sub_mesh arrays are greater than 0 and of the same length
    if len(sub_mesh_vertices) <= 0:
        print("There were no vertices in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) <= 0:
        print("There were no face in submesh to add, returning main mesh")
        return main_mesh_vertices, main_mesh_faces
    if len(sub_mesh_faces) != len(sub_mesh_vertices):
        raise Exception("The sub_mesh_faces and sub_mesh_vertices length did not match")
        
    
    #1) Count the number of vertices and faces in the main mesh
    n_main_vertices = len(main_mesh_vertices)
    n_main_faces = len(main_mesh_faces)
    
    
    #2) Iterate through the submesh vertices and faces. In loop:
    for i,(sub_verts, sub_faces) in enumerate(zip(sub_mesh_vertices,sub_mesh_faces)):
        #a. Count the number of vertices in the submesh and concate the vertices arrays to the main mesh array
        n_sub_verts = len(sub_verts)
        n_sub_faces = len(sub_faces)
        
        main_mesh_vertices = np.vstack([main_mesh_vertices,sub_verts])

        
        #b. Add the vertices_count of main to every number in the faces array
        sub_faces = sub_faces + n_main_vertices
        
        #c. Concatenate the submesh faces onto the larger mesh face
        main_mesh_faces = np.vstack([main_mesh_faces,sub_faces])
        
        #d. Save this new vertices and faces as the main_mesh verts and faces (DONE)
        
        #e. Print out how many new vertices and faces added
        #print(f"Added subsegment {i} with {n_sub_verts} vertices and {n_sub_faces} faces")
        
        n_main_vertices = len(main_mesh_vertices)
        n_main_faces = len(main_mesh_faces)
    
    #3) Print out number of segments added, total faces/vertices for new mesh  
    print(f"Added {len(sub_mesh_vertices)} subsegements \n  --> final mesh: {len(main_mesh_vertices)} vertices and {len(main_mesh_faces)} faces")
        
    return main_mesh_vertices,main_mesh_faces 

In [7]:
m65.FromNeuromancer()

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",version version of proofreading for the neuron,neuron_name Name of the neuron as it appears in the Neuromancer-Ingest website.,n_vertices,n_faces,vertices,faces
96631955273149705,0,RONNY,1659719,3161800,=BLOB=,=BLOB=
107816118160698192,0,CHARMAINE,937779,1791632,=BLOB=,=BLOB=


In [8]:
#1) Download the main mesh and all its subsegments and Concatenate the meshes according to the correct order
key = dict(segment_id=96631955273149705)
n = (m65.FromNeuromancer & key).fetch1()

print("---Main Neuron ---")
print(n["segment_id"])
print(n["n_vertices"])
print(n["vertices"])
print(n["faces"])

#get all of the segments and their data
lookup_key = dict(segment_id=n["segment_id"],version = n["version"])
subsegments = (m65.FromNeuromancer.Subsegment & lookup_key).fetch(as_dict=True)

subsegment_dicts = dict([(k["subsegment_id"],dict(vertices=k["vertices"],faces=k["faces"])) for k in subsegments])

subsegment_ordered_list = np.sort(np.array(list(subsegment_dicts.keys())))
subsegments_vertices = [subsegment_dicts[k]["vertices"] for k in subsegment_ordered_list]
subsegments_faces = [subsegment_dicts[k]["faces"] for k in subsegment_ordered_list]


new_mesh_vertices, new_mesh_faces = add_mesh_piece(main_mesh_vertices=n["vertices"],
           main_mesh_faces=n["faces"],
           sub_mesh_vertices = subsegments_vertices,
           sub_mesh_faces=subsegments_faces)

---Main Neuron ---
96631955273149705
1659719
[[ 848887.5     776986.25   1032329.625 ]
 [ 848993.      776912.875  1032339.8125]
 [ 848950.625   777083.0625 1032342.    ]
 ...
 [1083392.      758736.     1069720.    ]
 [1083392.      758160.     1070520.    ]
 [1083392.      758144.     1070540.    ]]
[[      0       2       1]
 [      0       1       3]
 [      4       6       5]
 ...
 [1656311 1655595 1657388]
 [1657331 1655945 1657332]
 [1658685 1657387 1658709]]
Added 80 subsegements 
  --> final mesh: 1862086 vertices and 3546312 faces


In [11]:
import trimesh
new_mesh = trimesh.Trimesh(vertices = new_mesh_vertices,faces=new_mesh_faces)

In [12]:
new_mesh.export(str(key["segment_id"]) + "_platinum_neuron.off")
print("hello")

hello


# Cleaning the Mesh of only the largest piece

In [13]:
new_mesh = trimesh.load_mesh("96631955273149705_Poisson_mesh.off")

In [15]:
mesh_splits = new_mesh.split(only_watertight=True)

In [26]:
len(mesh_splits)
#get the largest mesh
mesh_lengths = np.array([len(split.faces) for split in mesh_splits])

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()
# sns.distplot(mesh_lengths)

largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
largest_mesh = mesh_splits[largest_index][0]
largest_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [27]:
largest_mesh.export("96631955273149705_Poisson_mesh_largest_mesh.off")
print('hello')

hello
